## Refreshing the data
This relies on the `scraper.py` script being present and importable.

In [1]:
from scraper import scrape_to_csv

scrape_to_csv()

## Viewing the Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

In [2]:
tt_data = pd.read_csv('/data/time_tracking.csv', parse_dates=[0], index_col=[0])
# For now drop duplicates. Would be nice to investigate where these are coming from
tt_data = tt_data.drop_duplicates()
# sort everything by date
tt_data = tt_data.sort_index(0)
# To make life easier later, store column name for total packaging time as a constant
TOTAL_TIME = 'Total Time (1)'

## Selecting by Dates
The `tt_data` object is like a database table indexed by dates. Here are some examples of how to use that. Feel free to take a closer look at these slices of the data and play around with them or with new slices you create yourself.

In [3]:
# Selecting just one month
july = tt_data['7/2017']
# Selecting a range of months
may_to_july = tt_data['5/2017':'7/2017']

## Selecting By Project
You can also select items by project ID (first part of the Jira key).

In [21]:
itg = tt_data[tt_data['Project'] == "ITG"]

itg

,JIRA-Key,Project,Package Number,Name,Type,Complexity,Technology,Status,Packager,QA,Account/ Order#,Total Time (1),Time in period (2),QA passes,Overdue,Innovations (hr),Packaging & Development,Testing (hr),TR package (hr)
Date,,,,,,,,,,,,,,,,,,,
2012-01-01,ITG-1367,ITG,1367,KaratInnendienstclient_11-30-02.001,AppChange,Medium,NaN,Approved,s.nazarovskiy,s.nazarovskiy,NaN,9.0,9.0,1.0,NaN,NaN,5.5,3.5,NaN
2012-01-01,ITG-1368,ITG,1368,VIA-P_11-30-03.001,AppChange,Medium,MSI,Approved,s.nazarovskiy,v.lavrynenko,NaN,9.0,9.0,1.0,NaN,NaN,7.0,2.0,NaN
2012-01-01,ITG-1369,ITG,1369,JavaSE_1-6-0-30.001,AppChange,Medium,MSI,Approved,s.nazarovskiy,v.lavrynenko,NaN,8.5,8.5,1.0,NaN,NaN,5.5,3.0,NaN
2012-01-01,ITG-1370,ITG,1370,AcrobatReader_10-1-2.001,AppChange,Medium,MSI,Approved,m.kalachov,v.lavrynenko,NaN,9.0,9.0,1.0,NaN,NaN,6.0,3.0,NaN
2012-01-01,ITG-1371,ITG,1371,Office2003Patch_12-10.001,AppChange,Easy,Legacy setup,Approved,s.nazarovskiy,v.lavrynenko,NaN,4.5,4.5,1.0,NaN,NaN,2.5,2.0,NaN
2012-01-01,ITG-1372,ITG,1372,MSPatches-Client_12-10.001,AppChange,Complex,Legacy setup,Approved,s.nazarovskiy,v.lavrynenko,NaN,12.0,12.0,1.0,NaN,NaN,8.5,3.5,NaN
2012-01-01,ITG-1375,ITG,1375,Office2003-Konfiguration_12-10.001,AppChange,Medium,Legacy setup,Approved,d.kovalenko,v.lavrynenko,NaN,8.5,8.5,1.0,NaN,NaN,7.0,1.5,NaN
2012-01-01,ITG-1377,ITG,1377,PartnerDialog_12-10.001,AppChange,Medium,MSI,Approved,m.kalachov,v.lavrynenko,NaN,9.0,9.0,1.0,NaN,NaN,8.0,1.0,NaN
2012-01-01,ITG-1378,ITG,1378,PC-Text_12-10.001,AppChange,Medium,MSI,Approved,d.kovalenko,v.lavrynenko,NaN,7.5,7.5,1.0,NaN,NaN,6.0,1.5,NaN


## Duplicates

In [ ]:
# Just a demonstration of how many duplicate entries we have for an example package
erample_dupe = tt_data[(tt_data['Project'] == "NFUM") & (tt_data['Package Number'] == 480)]

example_dupe

## Packaging Times

In [4]:
by_complexity = tt_data.groupby('Complexity')[TOTAL_TIME]
by_complexity.describe()

,count,mean,std,min,25%,50%,75%,max
Complexity,,,,,,,,
Basic,6477.0,2.517693,1.875729,0.3,2.0,2.0,3.0,21.0
Complex,18939.0,17.262997,6.758269,0.3,14.9,19.0,20.3,67.0
Easy,35058.0,5.285367,1.957624,0.1,4.5,5.0,6.5,45.0
Medium,39933.0,9.698993,3.688607,0.2,8.0,10.0,12.0,70.0
NotSpecified,10617.0,2.967025,12.453282,0.1,1.0,1.5,3.0,327.5
Special,330.0,47.984545,45.037650,1.0,18.0,31.0,51.0,172.0


### Packaging time graphs

In [3]:
# Always run this cell, import and function used everywhere!

from ipywidgets import interact

from bokeh.io import push_notebook, output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, Span, Label


def _hline(y_value, x_values):
    """Generates a horizontal line by repeating y_value."""
    return np.repeat(y_value, len(x_values))

In [4]:
output_notebook()

Loading BokehJS ...

In [11]:
def generate_plot(data, title, height=500, width=900, planned_hours=2):

    p = figure(x_axis_type="datetime",
               plot_height=height,
               plot_width=width,
               active_scroll='wheel_zoom',
               title=title,
               title_location='above')

    xs = data.index.values
    tt = data[TOTAL_TIME]
    
    source = ColumnDataSource(dict(
        x = xs,
        y = tt,
        proj = data['Project'],
        pkg_id = data['Package Number']))    
    p.scatter('x', 
              'y', 
              name='datapoints', 
              source=source,
              legend="Total Time")

    p.line(xs, 
           tt.rolling(100, min_periods=1).mean(),
           line_width=3,
           color='black',
           name='rolling-mean',
           legend='Rolling Mean')
    p.line(xs,
           _hline(tt.mean(), xs),
           line_color='red',
           name='mean',
           legend="Avg Hours",
           line_width=2)
    p.line(xs,
           _hline(planned_hours, xs),
           line_color='green',
           name='planned',
           legend='Planned Hours',
           line_width=2)

#     This is currently tricky to add to the legend, keeping for reference
#     p.renderers.extend([
#         _hline(planned_hours, 'planned', 'green'),
#         _hline(tt.mean(), 'mean', 'red')])

#     The labels are supposed to explain the horizontal lines implemented by Spans
#     This is because Spans themselves can't be added to the legend for some reason.
#     my_label = Label(x=50, 
#                      y=planned_hours,
#                      x_units='screen',
#                      text='Planned Time',
#                      background_fill_color='white',
#                      background_fill_alpha=1.0)
#     p.add_layout(my_label)
    
#     my_label = Label(x=50, 
#                      y=tt.mean(),
#                      x_units='screen',
#                      text='Mean time',
#                      background_fill_color='white',
#                      background_fill_alpha=1.0)
#     p.add_layout(my_label)

    p.add_tools(HoverTool(
        tooltips=[("Project", "@proj"),
                  ('Package Number', '@pkg_id')]))

    return p

In [12]:
plot = generate_plot(tt_data[tt_data['Complexity'] == "Complex"], "Complex Packages", planned_hours=16)

show(plot, notebook_handle=True)

In [14]:
planned_times = {
    "Basic": 2,
    "Easy": 4,
    "Medium": 8,
    "Complex": 16
}

def update(plot):
    """Given a plot(figure) object returns a callback to be used by ipywidgets.interact."""
    def callback(complexity_type='Basic', project='All', rolling_window=100):
        pkg_filter = (tt_data['Complexity'] == complexity_type)
        if project is not 'All':
            pkg_filter = pkg_filter & (tt_data['Project'] == project)
        
        this_complexity = tt_data[pkg_filter]
        new_x = this_complexity.index.values
        new_tt = this_complexity[TOTAL_TIME]
        
        plot.select_one("datapoints").data_source.data = {
            'x': new_x,
            'y': new_tt,
            'proj': this_complexity['Project'],
            'pkg_id': this_complexity['Package Number']
        }
        plot.select_one("rolling-mean").data_source.data = {
            'x': new_x,
            "y": new_tt.rolling(rolling_window, min_periods=1).mean()
        }
        plot.select_one('planned').data_source.data = {
            "x": new_x,
            "y": np.repeat(planned_times[complexity_type], len(new_x))
        }
        plot.select_one('mean').data_source.data = {
            "x": new_x,
            "y": np.repeat(new_tt.mean(), len(new_x))
        }

        push_notebook()
    
    return callback

In [15]:
plot = generate_plot(tt_data, "Interactive")
show(plot, notebook_handle=True)

interact(update(plot),
         complexity_type=['Basic', 'Easy', 'Medium', 'Complex'],
         project=['All'] + list(tt_data['Project'].unique()),
         rolling_window=(5, 500)
)

<function __main__.update.<locals>.callback>

### Exporting to HTML

In [85]:
from bokeh.resources import CDN
from bokeh.embed import file_html, components

In [89]:
with open('index.html', 'w') as f:
    f.write(file_html(p, CDN, "Test Plot"))

In [36]:
script, div = components(p)

In [29]:
# I don't get what this says:
# http://bokeh.pydata.org/en/latest/docs/user_guide/server.html#python-callbacks-with-jupyter-interactors

## Selecting outliers

In [7]:
def select_outliers(df, complexity_type, value):
    this_complexity = df[df['Complexity'] == complexity_type]
    cutoff = this_complexity[TOTAL_TIME].std() + value
    return this_complexity[this_complexity[TOTAL_TIME] > cutoff]
    

In [8]:
len(select_outliers(tt_data, 'Easy', planned_times['Easy']))

11139